In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import loglikelihood as ll
from scipy.stats import t, norm

In [ ]:
df = pd.read_csv(r'C:\Users\joneh\master_thesis\data\time_series\CLc1_High_Frequency.csv')

df['LOGRET'] = np.log(df['CLOSE']).diff()

df.dropna(inplace=True)

display(df)

In [ ]:
def normal_loglikelihood(y, mu, sigma):
    return np.sum(norm.logpdf(y, loc=mu, scale=sigma))

def t_likelihood(y, mu, sigma, df):
    return np.sum(t.logpdf(y, df, loc=mu, scale=sigma))

def heavyr(omega: float, alpha: float, beta: float, rm_last: float, h_last: float) -> float:
    h = omega + alpha * rm_last + beta * h_last
    return h

def heavyrm(omega: float, alpha: float, beta: float, rm_last: float, mu_last: float) -> float:
    h = omega + alpha * rm_last + beta * mu_last
    return h

def heavyr_likelihood(
        params: dict[str:float], 
        r2_data:np.ndarray, 
        rm_data:np.ndarray, 
        h0: float, 
        lag: int=1, 
        tstudent_df: float=0.0, 
        estimating: bool=True
    ) -> any:

    t = len(r2_data)
    h = np.repeat(h0, t)

    for i in range(lag, t):
        h[i] = heavyr(params['omega'], params['alpha'], params['beta'], rm_data[i-lag], h[i-lag])

    if estimating:
        return ll.generic_loglikelihood(r2_data, h, df=tstudent_df, sign=-1.0)
    
    else:
        return ll.generic_loglikelihood(r2_data, h, df=tstudent_df, sign=1.0), np.copy(h)
    

def heavy_forecast(
    h_params: dict[str:float],
    mu_params: dict[str:float],
    rm_past: np.ndarray,
    h_past: np.ndarray,
    mu_past: np.ndarray,
    steps: int=1,
    full_output: bool=False    
    ) -> np.ndarray:

    lag = len(rm_past)
    h = np.zeros(steps)
    mu = np.zeros(steps)

    for i in range(0, steps):
        if i < lag:
            h[i] = heavyr(h_params['omega'], h_params['alpha'], h_params['beta'], rm_past[i], h_past[i])

            mu[i] = heavyrm(mu_params['omega'], mu_params['alpha'], mu_params['beta'], rm_past[i], mu_past[i])

        else:
            h[i] = heavyr(h_params['omega'], h_params['alpha'], h_params['beta'], rm_past[i - lag], h[i - lag])

            mu[i] = heavyrm(mu_params['omega'], mu_params['alpha'], mu_params['beta'], rm_past[i - lag], mu[i - lag])

    if full_output:
        return np.copy(h), np.copy(mu)
    else:
        return np.copy(h)


# use the first 1000 observations to estimate the parameters
t = 1000
r2_data = df['LOGRET'].values[:t]

# initial values
h0 = 0.0001

params = {'alpha': 0.1, 'beta': 0.8, 'omega': 0.0001}

# estimate the parameters
result = heavyr_likelihood(params, r2_data, r2_data, h0, estimating=True)